# Keras

- Keras is the high-level API of the TensorFlow platform. It provides an approachable, highly-productive interface for solving machine learning (ML) problems, with a focus on modern deep learning. Keras covers every step of the machine learning workflow, from data processing to hyperparameter tuning to deployment. It was developed with a focus on enabling fast experimentation.

- Keras is designed to reduce cognitive load by achieving the following goals:
    - Offer simple, consistent interfaces.
    - Minimize the number of actions required for common use cases.
    - Provide clear, actionable error messages.
    -Follow the principle of progressive disclosure of complexity: It's easy to get started, and you can complete advanced workflows by learning as you go.
    - Help you write concise, readable code.


# Keras API components
The core data structures of Keras are layers and models.
- **layery**: 
    - A layer is a simple input/output transformation 
    - The tf.keras.layers.Layer class is the fundamental abstraction in Keras. A Layer encapsulates a state (weights) and some computation (defined in the tf.keras.layers.Layer.call method).
    - You can also use layers to handle data preprocessing tasks like normalization and text vectorization.
    - Preprocessing layers can be included directly into a model, either during or after training, which makes the model portable.
    
- **models**: 
    - A model is a directed acyclic graph (DAG) of layers.
    - A model is an object that groups layers together and that can be trained on data.

The tf.keras.Model class features built-in training and evaluation methods:
   - **tf.keras.Model.fit**: Trains the model for a fixed number of epochs.
   - **tf.keras.Model.predict**: Generates output predictions for the input samples.
   - **tf.keras.Model.evaluate**: Returns the loss and metrics values for the model; configured via the tf.keras.Model.compile method.
   
These methods give you access to the following built-in training features:
   - **Callbacks**: You can leverage built-in callbacks for early stopping, model checkpointing, and TensorBoard monitoring. You can also implement custom callbacks.
   - **Distributed training**: You can easily scale up your training to multiple GPUs, TPUs, or devices.
   - **Step fusing**: With the steps_per_execution argument in tf.keras.Model.compile, you can process multiple batches in a single tf.function call, which greatly improves device utilization on TPUs.




# Building simple model with keras.layers.Layer

- **tf.keras.layers.Layer** is the base class of all Keras layers, and it inherits from **tf.Module**.

In [1]:
import tensorflow as tf

2023-07-16 07:18:57.795969: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 07:18:57.993237: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 07:18:57.994207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 07:18:59.055312: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [27]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, in_features, out_features, **kwargs):
        super().__init__(**kwargs)
        
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name="w")
        self.b = tf.Variable(tf.zeros([out_features]), name="b")
    
    def call(self, x): # note that instead of using __call__ like in tf.Module, we are just using call() method
        z = x @ self.w + self.b
        return tf.nn.relu(z)

In [28]:
simple_layer = MyDense(3, 2)
simple_layer.variables

[<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
 array([[-0.6004024 , -0.97726893],
        [-0.7292261 ,  0.8978605 ],
        [-0.39256954,  0.7489062 ]], dtype=float32)>,
 <tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [29]:
simple_layer([[1.0 , 1.0, 2.0],
              [2.0, 2.0, 1.0]])

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.       , 1.418404 ],
       [0.       , 0.5900894]], dtype=float32)>

### with build step
- As noted, it's convenient in many cases to wait to create variables until you are sure of the input shape.
- build is called exactly once, and it is called with the shape of the input. It's usually used to create variables (weights).

In [122]:
class FlexibleDense(tf.keras.layers.Layer):
    def __init__(self, out_features, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features
    
    def build(self, input_shape):
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]), name="w")
        self.b = tf.Variable(tf.zeros([self.out_features]), name="b")
    
    def call(self, x):
        y = x @ self.w + self.b
        return tf.nn.relu(y)

In [123]:
flexible_dense = FlexibleDense(out_features=2)

In [124]:
flexible_dense.weights

[]

Weights are not initialized yet, It won't be aviable not until the "call" method will be called.

In [125]:
x = tf.constant([[1.0, 2.0, 3.0]])
x.shape

TensorShape([1, 3])

In [126]:
flexible_dense(x)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.       , 1.1477716]], dtype=float32)>

In [127]:
flexible_dense.variables

[<tf.Variable 'flexible_dense_43/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[-1.9676659 ,  0.64514977],
        [-0.37998724,  0.41509008],
        [ 0.29196718, -0.10918608]], dtype=float32)>,
 <tf.Variable 'flexible_dense_43/b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

As we can see, weights are initialized now. [source code](https://github.com/keras-team/keras/blob/e327db2f7016e3605593f6687e48daf815391a7f/keras/engine/base_layer.py)

## Keras model
- We can define model as a nested keras layers (using base class **keras.layers.Layers**), like we used tf.Module to define model as well as it's layers.
- However, Keras also provides a full-featured model class called **tf.keras.Model**. It inherits from **tf.keras.layers.Layer**, so a Keras model can be used and nested in the same way as Keras layers. 

- class inheritances:
    - **tf.Module --> keras.layers.Layers ---> keras.Modle**
    - notation: base-class --> derived_class

In [133]:
class MySequentialModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.dense1 = FlexibleDense(out_features=4)
        self.dense2 = FlexibleDense(out_features=2)
    
    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [136]:
model = MySequentialModel(name="simplest model")

In [137]:
# singel weights are not yet initialized, it will throw an exception
try:
    model.summary()
except Exception as e:
    print(e)

This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.


In [ ]:
model(tf.constant([[1., 2., 3.]]))

In [131]:
model.submodules

(<__main__.FlexibleDense at 0x7fa738187c40>,
 <__main__.FlexibleDense at 0x7fa7307a0cd0>)

In [132]:
model.summary()

Model: "simplest model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flexible_dense_44 (Flexible  multiple                 16        
 Dense)                                                          
                                                                 
 flexible_dense_45 (Flexible  multiple                 10        
 Dense)                                                          
                                                                 
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________
